In [117]:
import pandas as pd
import numpy as np
from itertools import chain

In [95]:
df = pd.read_csv('prepost_matched_debt_files.csv', index_col = 0)

In [96]:
def stringToList(x):
    return x.strip('][\"\'').split(', ') if not pd.isnull(x) else x

In [97]:
df['full name'] = df['full name'].apply(lambda x: stringToList(x))
df['Same State Loan Office'] = df['Same State Loan Office'].apply(lambda x: 
                                                                  stringToList(x))
df['Different State Loan Office'] = df['Different State Loan Office'].apply(lambda x: 
                                                                            stringToList(x))
df['Same State Liquidated Debt'] = df['Same State Liquidated Debt'].apply(lambda x: 
                                                                          stringToList(x))
df['Different State Liquidated Debt'] = df['Different State Liquidated Debt'].apply(lambda x: 
                                                                                    stringToList(x))
df['Pierce Certificates'] = df['Pierce Certificates'].apply(lambda x: 
                                                            stringToList(x))

# How many unique individuals were issued 6 percent stocks and deferred 6 percent stocks in 1790 and after?

In [98]:
df.groupby('state')['full name'].count()

state
CT    747
GA     54
MD    356
NC     54
NH    169
NJ    566
NY    882
PA    839
RI    502
SC    274
VA    509
Name: full name, dtype: int64

# How many of these individuals
- were original purchasers of loan office certicates of the same state as the 6 percent stock?
- were original purchasers of loan office certicates issued from another state?
- were original recipients of liquidated debt certicates issued by the same-state loan office? other state loan offices?
- were original recipients of the Pierce Certicates?

### How many individuals were original purchasers of loan office certicates of the same state as the 6 percent stock?

In [99]:
df[df['Same State Loan Office'].apply(lambda x: 
                                      type(x) == list)]['state'].value_counts()

CT    327
PA    307
MD    102
VA     82
NH     63
NY     56
NJ     24
RI     15
SC     14
GA      2
Name: state, dtype: int64

### How many individuals were original purchasers of loan office certicates of a different state as the 6 percent stock?

In [100]:
df[df['Different State Loan Office'].apply(lambda x: 
                                           type(x) == list)]['state'].value_counts()

PA    128
CT    115
VA    103
MD     59
NY     44
NH     42
NJ     30
SC     18
RI     16
GA      2
NC      1
Name: state, dtype: int64

##### Aside: How many individuals were original purchasers of loan office certicates of a different and same state as the 6 percent stock?

In [101]:
df[[type(ss) == list and type(ds) == list for 
    ss, ds in zip(df['Same State Loan Office'], 
                  df['Different State Loan Office'])]]['state'].value_counts()

PA    84
CT    67
VA    39
MD    21
NJ    16
NH    14
NY    14
SC    10
RI     8
GA     2
Name: state, dtype: int64

### How many individuals were original recipients of liquidated debt certificates issued by the same state loan office?

In [102]:
df[df['Same State Liquidated Debt'].apply(lambda x: 
                                          type(x) == list)]['state'].value_counts()

PA    213
CT    107
NY     51
NH     29
VA     14
NJ      9
SC      7
RI      6
MD      3
GA      1
Name: state, dtype: int64

### How many individuals were original recipients of liquidated debt certificates issued by another state loan office?

In [103]:
df[df['Different State Liquidated Debt'].apply(lambda x: 
                                               type(x) == list)]['state'].value_counts()

CT    185
PA    131
NY     74
NH     59
VA     59
MD     41
NJ     34
RI     21
SC     15
GA      1
Name: state, dtype: int64

##### Aside: How many individuals were original purchasers of liquidated debt certificates from the same and another loan office?

In [104]:
df[[type(ss) == list and type(ds) == list for 
    ss, ds in zip(df['Same State Liquidated Debt'], 
                  df['Different State Liquidated Debt'])]]['state'].value_counts()

CT    107
PA     65
NY     48
NH     29
VA     13
NJ      8
RI      6
SC      5
MD      3
GA      1
Name: state, dtype: int64

### How many individuals were original recipients of the Pierce Certicates, corresponding to the same state?

In [105]:
df[df['Pierce Certificates'].apply(lambda x: 
                                   type(x) == list)]['state'].value_counts()

CT    182
PA    166
VA    146
MD     82
NY     55
NH     35
NJ     23
SC     16
RI     15
NC      2
GA      1
Name: state, dtype: int64

### For how many individuals do the face values of their pre-1790 certificates equal the sum of the face values of the 6 percent and deferred 6 percent stocks that they received?

##### Number of people with non-zero assets and equal values

In [106]:
df[[tt == dt and tt != 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]].shape[0]

14

##### State Distribution

In [107]:
df[[tt == dt and tt != 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]]['state'].value_counts()

PA    4
MD    3
CT    3
VA    2
NH    2
Name: state, dtype: int64

##### Number of people with zero assets and equal values
Most of these occur when someone has 3% but no 6% certificates

In [108]:
df[[tt == dt and tt == 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]].shape[0]

1299

##### State Distribution

In [109]:
df[[tt == dt and tt == 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]]['state'].value_counts()

NY    688
NJ    525
VA     26
CT     23
RI     17
PA      9
MD      8
SC      3
Name: state, dtype: int64

### Who, and how many people redeemed certificates with Hamilton and were not original owners/purchasers

In [110]:
print(df[df['tot_pre1790_certs'] == 0]['state'].shape[0])

3249


##### state distribution

In [111]:
df[df['tot_pre1790_certs'] == 0]['state'].value_counts()

NY    757
NJ    520
RI    473
PA    357
VA    286
CT    269
SC    249
MD    180
NH     55
GA     52
NC     51
Name: state, dtype: int64

In [138]:
namelsts = df[df['tot_pre1790_certs'] == 0]['full name'].apply(lambda x: pd.Series(x)).values
names = [x for x in list(chain.from_iterable(namelsts)) if not pd.isnull(x)]

In [188]:
name_state_asset = pd.DataFrame({}, columns = ['Full Name', 'Total', 'state'])

In [189]:
name_df = df[df['tot_pre1790_certs'] == 0]['full name'].apply(lambda x: pd.Series(x))
for name in names:
    ind1 = name_df[name_df[0] == name].index.tolist()
    ind2 = name_df[name_df[1] == name].index.tolist()
    ind3 = name_df[name_df[2] == name].index.tolist()
    ind1.extend(ind2)
    ind1.extend(ind3)
    ind1 = list(set(ind1))
    tot_assets = df.loc[ind1, 'Total'].sum()
    state = df.loc[ind1, 'state'].values[0]
    add_df = pd.DataFrame()
    add_df['Full Name'] = [name]
    add_df['Total'] = [tot_assets]
    add_df['state'] = [state]
    name_state_asset = pd.concat([name_state_asset, add_df])

In [199]:
cd_names = pd.DataFrame(name_state_asset.groupby(['Full Name', 'state'])['Total'].sum()).reset_index()

In [201]:
cd_names.to_csv("speculator_names.csv")